In [1]:
!python3 --version
!pip3 list

Python 3.8.10
Package                 Version
----------------------- --------------------
absl-py                 0.13.0
anyio                   3.2.1
appdirs                 1.4.3
apturl                  0.5.2
argon2-cffi             20.1.0
astunparse              1.6.3
async-generator         1.10
attrs                   21.2.0
autopep8                1.5.7
Babel                   2.9.1
backcall                0.2.0
beautifulsoup4          4.9.3
bitarray                2.3.3
bleach                  3.3.0
blinker                 1.4
Brlapi                  0.7.0
cachetools              4.2.2
certifi                 2019.11.28
cffi                    1.14.5
chardet                 3.0.4
chrome-gnome-shell      0.0.0
clang                   5.0
Click                   7.0
colorama                0.4.3
command-not-found       0.3
cryptography            2.8
cupshelpers             1.0
cycler                  0.10.0
dbus-python             1.2.16
decorator               5.0.9
defer      

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import warnings
import os
import pandas as pd
import numpy as np
import modules.model as Model
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

warnings.filterwarnings('ignore')

In [4]:
X_train, y_train = Model.loadData("./data/data/train")
X_train = X_train.fillna("")

In [5]:
display(X_train, y_train)

,raw_comment,normalize_comment,emoji_decode
0,form k đẹp lắm,form không đẹp lắm,
1,Áo Rộng thật sự\nGl nhanh\nChật lươ...,áo rộng thật sự nhanh chật lượng v...,
2,Màu túi hơi tối do với ảnh chụp,màu túi hơi tối do ảnh chụp,
3,Chất liệu vải k ổn lắm,chất liệu vải không ổn lắm,
4,"Mã ""màu đen"" hiện lên áo màu đen, còn ...",mã màu đen hiện lên áo màu đen còn mã...,
...,...,...,...
11359,"Khi bán combo ghi 3 đôi, nhưng nhận chỉ ...",bán combo ghi đôi nhưng nhận chỉ đôi ta...,
11360,"Áo đẹp, from chuẩn. Sẽ ủng hộ tiếp ạ !",áo đẹp from chuẩn sẽ ủng hộ tiếp,
11361,Đặt màu trắng kem thì giao màu xanh đen...,đặt màu trắng kem giao màu xanh đen giao...,expressionless
11362,Đồ chơi tí hon.\nShop nên dừng bán sp ...,đồ chơi tí hon nên dừng bán sản phẩ...,


,label
0,0
1,1
2,0
3,0
4,0
...,...
11359,0
11360,1
11361,0
11362,0


In [6]:
ngram_words = Model.getDashWords(X_train['normalize_comment'], [2, 3, 4])

In [7]:
ngram_words

,freq_doc,freq
giao hàng,1914,1914
sản phẩm,1803,1803
chất lượng,1377,1377
hàng nhanh,1082,1082
giao hàng nhanh,1027,1027
...,...,...
sáp không khô môi,1,1
không khô môi lưu,1,1
khô môi lưu màu,1,1
môi lưu màu cũng,1,1


In [8]:
print(len(ngram_words[ngram_words['freq_doc'] >= 10]))
print(len(ngram_words[ngram_words['freq_doc'] >= 20]))

3755
1540


In [9]:
X_train['dash_comment'] = Model.replaceInNGrams(X_train['normalize_comment'], [2, 3, 4], ngram_words, 'freq_doc')
X_train['dash_comment_min_20'] = Model.replaceInNGrams(X_train['normalize_comment'], [2, 3, 4], ngram_words, 'freq_doc', 20)

In [10]:
X_train

,raw_comment,normalize_comment,emoji_decode,dash_comment,dash_comment_min_20
0,form k đẹp lắm,form không đẹp lắm,,form_không đẹp_lắm,form_không đẹp_lắm
1,Áo Rộng thật sự\nGl nhanh\nChật lươ...,áo rộng thật sự nhanh chật lượng v...,,áo_rộng thật_sự nhanh_chật_lượng v...,áo_rộng thật_sự nhanh chật lượng v...
2,Màu túi hơi tối do với ảnh chụp,màu túi hơi tối do ảnh chụp,,màu túi_hơi tối_do ảnh_chụp,màu túi hơi tối do ảnh chụp
3,Chất liệu vải k ổn lắm,chất liệu vải không ổn lắm,,chất_liệu vải_không ổn_lắm,chất_liệu vải_không ổn lắm
4,"Mã ""màu đen"" hiện lên áo màu đen, còn ...",mã màu đen hiện lên áo màu đen còn mã...,,mã màu_đen hiện lên_áo màu_đen còn_mã...,mã màu_đen hiện lên áo màu_đen còn mã...
...,...,...,...,...,...
11359,"Khi bán combo ghi 3 đôi, nhưng nhận chỉ ...",bán combo ghi đôi nhưng nhận chỉ đôi ta...,,bán_combo ghi_đôi nhưng_nhận chỉ đôi_ta...,bán combo ghi đôi nhưng_nhận chỉ đôi_ta...
11360,"Áo đẹp, from chuẩn. Sẽ ủng hộ tiếp ạ !",áo đẹp from chuẩn sẽ ủng hộ tiếp,,áo_đẹp from_chuẩn sẽ ủng_hộ tiếp,áo_đẹp from chuẩn sẽ ủng_hộ tiếp
11361,Đặt màu trắng kem thì giao màu xanh đen...,đặt màu trắng kem giao màu xanh đen giao...,expressionless,đặt_màu trắng_kem giao_màu xanh_đen giao...,đặt_màu trắng kem giao_màu xanh đen giao...
11362,Đồ chơi tí hon.\nShop nên dừng bán sp ...,đồ chơi tí hon nên dừng bán sản phẩ...,,đồ_chơi tí_hon_nên_dừng bán sản_phẩ...,đồ chơi tí hon nên dừng bán sản_phẩ...


In [11]:
vectorizers = [
    ("Bag of Words", Model.vectorizer(X_train['dash_comment'], 'bow')),
    ("TF-IDF", Model.vectorizer(X_train['dash_comment'], 'tfidf')),
    ("Bag of Words- [min_df: 5]", Model.vectorizer(X_train['dash_comment'], 'bow', 5)),
    ("TF-IDF- [min_df: 5]", Model.vectorizer(X_train['dash_comment'], 'tfidf', 5)),
    ("Bag of Words - [min_df: 10]", Model.vectorizer(X_train['dash_comment'], 'bow', 10)),
    ("TF-IDF - [min_df: 10]", Model.vectorizer(X_train['dash_comment'], 'tfidf', 10)),
    ("Bag of Words - [min_df: 20]", Model.vectorizer(X_train['dash_comment'], 'bow', 20)),
    ("TF-IDF - [min_df: 20]", Model.vectorizer(X_train['dash_comment'], 'tfidf', 20))
]

ix = 0
print(len(vectorizers[ix][1][0].get_feature_names()))
print(vectorizers[ix][1][0].get_feature_names()[:100])
pd.DataFrame(data=vectorizers[ix][1][1].toarray(), columns=vectorizers[ix][1][0].get_feature_names())

20010
['a_a', 'a_av', 'a_bê', 'a_cho', 'a_cùng', 'a_e', 'a_hong', 'a_hời', 'a_rất', 'a_xã', 'ac_clone', 'ace', 'ad', 'ad_bọc', 'ad_gửi', 'ad_vào', 'ah', 'ah_ai', 'ah_nhen', 'ai', 'ai_chân', 'ai_có', 'ai_cũng', 'ai_cả', 'ai_cần', 'ai_dám', 'ai_dè', 'ai_eo', 'ai_không', 'ai_luôn', 'ai_làm', 'ai_mua', 'ai_muốn', 'ai_mà', 'ai_mất', 'ai_mặc', 'ai_nghĩ', 'ai_ngờ', 'ai_nhận', 'ai_nhắn', 'ai_review', 'ai_rãnh', 'ai_rộng', 'ai_tay', 'ai_thích', 'ai_trắng', 'ai_về', 'ai_đời', 'am', 'amazing', 'an', 'an_lành', 'an_toàn', 'and', 'and_tacos', 'and_think', 'android_gồm', 'anh', 'anh_chủ', 'anh_cute', 'anh_có', 'anh_de', 'anh_em', 'anh_giữ', 'anh_không', 'anh_muốn', 'anh_nên', 'anh_nói', 'anh_ship', 'anh_siêu', 'anh_đáng', 'anh_ơi', 'anh_ấy', 'any_nên', 'ao', 'ao_cũng', 'ao_kha', 'ao_khác', 'ao_như', 'ao_thi', 'ao_thun', 'ao_vua', 'auto', 'auto_hay', 'b_có', 'b_nào', 'b_nên', 'ba', 'ba_cai', 'ba_chấm', 'ba_con', 'ba_cái', 'ba_lô', 'ba_tia', 'ba_đôi', 'bai', 'bai_giãn', 'bam', 'ban', 'ban_cái']


,a_a,a_av,a_bê,a_cho,a_cùng,a_e,a_hong,a_hời,a_rất,a_xã,...,ủng_hộc,ức_chế,ức_x,ức_óc_ức_of,ứng_dụng_trên_android,ứng_luôn,ừ_mười,ừ_ừ,ửng,ửng_hộ
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11359,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11360,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11361,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11362,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
ix = 3
print(len(vectorizers[ix][1][0].get_feature_names()))
print(vectorizers[ix][1][0].get_feature_names()[:100])
pd.DataFrame(data=vectorizers[ix][1][1].toarray(), columns=vectorizers[ix][1][0].get_feature_names())

2742
['ah', 'ai', 'ai_có', 'ai_cũng', 'ai_dè', 'ai_mua', 'ai_ngờ', 'anh', 'anh_em', 'ao', 'ban_đầu', 'bao', 'bao_bì', 'bao_giờ', 'bao_lâu', 'bao_nhiêu', 'bay_màu', 'be', 'biết', 'biết_là', 'bong_tróc', 'bung', 'bung_chỉ', 'bung_hết', 'bung_ra', 'buôn_bán', 'buồn', 'buồn_ghê', 'buồn_quá', 'buồn_vì', 'buộc_tóc', 'bàn_tay', 'bán', 'bán_hàng', 'bán_đắt', 'bán_đồ', 'báo', 'báo_giao', 'báo_khách', 'bát_gạo', 'bây_giờ', 'bé', 'bé_hơn', 'bé_quá', 'bé_tí', 'bé_xíu', 'bên', 'bên_hông', 'bên_ngoài', 'bên_trong', 'bình_luận', 'bình_thường', 'bò_nhạt', 'bò_đậm', 'bó', 'bó_tay', 'bóc_ra', 'bóng', 'bóp', 'bông_tai', 'bù', 'băng_đô', 'bạc', 'bạc_màu', 'bạn', 'bạn_bè', 'bạn_nào', 'bảng_size', 'bảo', 'bảo_gửi', 'bảo_hành', 'bảo_sẽ', 'bấm', 'bất_ngờ', 'bất_tiện', 'bẩn', 'bận', 'bằng', 'bằng_nhau', 'bằng_nhựa', 'bền', 'bền_không', 'bọc', 'bọc_hàng', 'bỏ', 'bỏ_luôn', 'bỏ_qua', 'bỏ_ra', 'bỏ_tiền', 'bỏ_vừa', 'bố', 'bộ', 'bộ_còn', 'bộ_hồng', 'bộ_đồ', 'bục', 'bục_chỉ', 'bụng', 'bực', 'cam']


,ah,ai,ai_có,ai_cũng,ai_dè,ai_mua,ai_ngờ,anh,anh_em,ao,...,ổn_so,ổn_trong,ổn_áp,ổn_đẹp,ở_ngoài,ở_nhà,ở_trong,ở_trên,ở_đây,ủng_hộ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
11360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.42178
11361,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
11362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000


In [13]:
ix = 5
print(len(vectorizers[ix][1][0].get_feature_names()))
print(vectorizers[ix][1][0].get_feature_names()[:100])
pd.DataFrame(data=vectorizers[ix][1][1].toarray(), columns=vectorizers[ix][1][0].get_feature_names())

1454
['ah', 'ai', 'ai_ngờ', 'anh', 'anh_em', 'ao', 'ban_đầu', 'bao', 'bao_giờ', 'bao_nhiêu', 'bay_màu', 'biết', 'biết_là', 'bung_chỉ', 'bung_hết', 'buôn_bán', 'buồn', 'buộc_tóc', 'bán', 'bán_hàng', 'bán_đắt', 'báo', 'báo_giao', 'bát_gạo', 'bé', 'bé_hơn', 'bên', 'bên_ngoài', 'bên_trong', 'bình_luận', 'bình_thường', 'bó_tay', 'bóc_ra', 'bóng', 'bóp', 'bông_tai', 'bù', 'bạn', 'bạn_nào', 'bảo', 'bảo_hành', 'bất_ngờ', 'bẩn', 'bận', 'bằng', 'bền', 'bền_không', 'bọc', 'bỏ', 'bỏ_ra', 'bỏ_tiền', 'bố', 'bộ', 'bộ_đồ', 'bục_chỉ', 'bụng', 'bực', 'can_than', 'cao', 'cao_mặc', 'cao_nặng', 'chat', 'chi_tiết', 'chim_ưng', 'chiếc', 'chiều_dài', 'cho', 'cho_cái', 'cho_có', 'cho_khách', 'cho_luôn', 'cho_lắm', 'cho_người', 'cho_vì', 'cho_đổi', 'chu_đáo', 'chuyên_nghiệp', 'chuyển_khoảng', 'chuẩn', 'chuẩn_hàng', 'chán', 'chán_lắm', 'chán_quá', 'chân', 'chân_váy', 'chú_ý', 'chúc', 'chúc_buôn', 'chút', 'chút_nhưng', 'chút_nào', 'chăm_sóc', 'chưa', 'chưa_biết', 'chưa_dùng', 'chưa_mặc', 'chưa_thấy', 'chưa_được',

,ah,ai,ai_ngờ,anh,anh_em,ao,ban_đầu,bao,bao_giờ,bao_nhiêu,...,ổn,ổn_lắm,ổn_nhưng,ổn_so,ổn_áp,ở_ngoài,ở_nhà,ở_trong,ở_đây,ủng_hộ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.685498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
11360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.42178
11361,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
11362,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000


In [14]:
ix = 7
print(len(vectorizers[ix][1][0].get_feature_names()))
print(vectorizers[ix][1][0].get_feature_names()[-100:])
pd.DataFrame(data=vectorizers[ix][1][1].toarray(), columns=vectorizers[ix][1][0].get_feature_names())

754
['xong', 'xuất_sắc', 'xám', 'xíu', 'xù_lông', 'xấu', 'xấu_không', 'xịn', 'xịn_xò', 'y_hình', 'yên_tâm', 'yêu_cầu', 'áo', 'áo_chất', 'áo_có', 'áo_cũng', 'áo_hơi', 'áo_không', 'áo_mặc', 'áo_mỏng', 'áo_ok', 'áo_quá', 'áo_rất', 'áo_thun', 'áo_trắng', 'áo_vải', 'áo_xinh', 'áo_đẹp', 'ý_kiến', 'đen', 'đeo', 'đi', 'đáng', 'đáng_mua', 'đáng_tiền', 'đánh_giá', 'đây', 'đây_là', 'đòi_hỏi', 'đóng_gói', 'đôi', 'đôi_tất', 'đúng', 'đúng_là', 'đúng_màu', 'đúng_mẫu', 'đúng_như', 'đúng_size', 'đơn', 'đơn_hàng', 'đường_chỉ', 'đường_may', 'được', 'được_cái', 'được_giá', 'được_nhưng', 'được_tặng', 'đấy', 'đầu', 'đầy_đủ', 'đặt', 'đặt_cái', 'đặt_hàng', 'đặt_màu', 'đặt_size', 'đặt_xl', 'đặt_áo', 'đẹp', 'đẹp_chất', 'đẹp_hơn', 'đẹp_luôn', 'đẹp_lắm', 'đẹp_nhưng', 'đẹp_vải', 'đến', 'đều', 'đồ', 'đồng_hồ', 'đồng_tiền', 'đổi', 'đổi_hàng', 'đổi_size', 'đổi_trả', 'đợi', 'đủ_hàng', 'đừng', 'đựng_được', 'ơi', 'ưng', 'ưng_luôn', 'ưng_lắm', 'ưng_ý', 'ảnh', 'ảnh_không', 'ấy', 'ổn', 'ở_ngoài', 'ở_nhà', 'ở_đây', 'ủng_hộ'

,ai,bao_giờ,bao_nhiêu,biết,bung_chỉ,buộc_tóc,bán,bán_hàng,bán_đắt,báo,...,ưng_lắm,ưng_ý,ảnh,ảnh_không,ấy,ổn,ở_ngoài,ở_nhà,ở_đây,ủng_hộ
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11359,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
11360,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.42178
11361,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000
11362,0.0,0.0,0.0,0.0,0.0,0.0,0.539375,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000


In [15]:
lst_models = [
    ('Logistic Regression - [solver: lbfgs]', LogisticRegression(solver='lbfgs')),
    ('Logistic Regression - [solver: liblinear]', LogisticRegression(solver='liblinear')),
    ('Logistic Regression - [solver: newton-cg]', LogisticRegression(solver='newton-cg')),
    ('KNN - [n_neighbors: 2]', KNeighborsClassifier(n_neighbors=2)),
    ('KNN - [n_neighbors: 3]', KNeighborsClassifier(n_neighbors=3)),
    ('SVC - [kernel: linear]', SVC(kernel='linear', random_state=42)),
    ('SVC - [kernel: poly]', SVC(kernel='poly', random_state=42)),
    ('SVC - [kernel: rbf]', SVC(kernel='rbf', random_state=42)),
    ('SVC - [kernel: sigmoid]', SVC(kernel='sigmoid', random_state=42)),
    ('Bernoulli', BernoulliNB()),
    ('Random Forest', RandomForestClassifier(random_state=42)),
    ('XGBoost', XGBClassifier(eval_metric='mlogloss'))
]

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [16]:
original_models = Model.train(lst_models, vectorizers[2:], y_train, cv)

original_models

Bag of Words- [min_df: 5]:
	Model Logistic Regression - [solver: lbfgs] has been trained in 0.96 seconds
	Model Logistic Regression - [solver: liblinear] has been trained in 0.39 seconds
	Model Logistic Regression - [solver: newton-cg] has been trained in 1.02 seconds
	Model KNN - [n_neighbors: 2] has been trained in 31.24 seconds
	Model KNN - [n_neighbors: 3] has been trained in 33.71 seconds
	Model SVC - [kernel: linear] has been trained in 81.84 seconds
	Model SVC - [kernel: poly] has been trained in 165.82 seconds
	Model SVC - [kernel: rbf] has been trained in 141.31 seconds
	Model SVC - [kernel: sigmoid] has been trained in 78.21 seconds
	Model Bernoulli has been trained in 0.15 seconds
	Model Random Forest has been trained in 55.12 seconds
	Model XGBoost has been trained in 3.89 seconds
TF-IDF- [min_df: 5]:
	Model Logistic Regression - [solver: lbfgs] has been trained in 0.58 seconds
	Model Logistic Regression - [solver: liblinear] has been trained in 0.27 seconds
	Model Logistic

,vectorizer,model,train_acc,test_acc,diff_acc,train_acc_std,test_acc_std,train_roc_auc,test_roc_auc,diff_roc_auc,train_roc_auc_std,test_roc_auc_std,fit_time
0,TF-IDF- [min_df: 5],SVC - [kernel: rbf],0.960431,0.860610,0.099820,0.000862,0.006475,0.988109,0.920209,0.067900,0.000571,0.006300,5.011927
1,TF-IDF- [min_df: 5],SVC - [kernel: linear],0.909793,0.859379,0.050414,0.001363,0.007170,0.964620,0.926554,0.038066,0.000480,0.005974,3.164958
2,Bag of Words- [min_df: 5],Bernoulli,0.881742,0.858763,0.022979,0.001342,0.011235,0.947202,0.925786,0.021416,0.000517,0.006771,0.003467
3,TF-IDF- [min_df: 5],Bernoulli,0.881742,0.858763,0.022979,0.001342,0.011235,0.947202,0.925786,0.021416,0.000517,0.006771,0.003507
4,TF-IDF- [min_df: 5],Logistic Regression - [solver: liblinear],0.896603,0.858323,0.038280,0.001177,0.006163,0.959189,0.929278,0.029911,0.000363,0.005642,0.018142
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,TF-IDF - [min_df: 20],KNN - [n_neighbors: 2],0.777690,0.669394,0.108296,0.001445,0.009342,0.944053,0.739783,0.204269,0.002827,0.016440,0.002527
68,TF-IDF - [min_df: 10],KNN - [n_neighbors: 3],0.769114,0.654967,0.114147,0.074013,0.014413,0.936307,0.730836,0.205471,0.006121,0.014794,0.002572
69,TF-IDF - [min_df: 10],KNN - [n_neighbors: 2],0.725683,0.630235,0.095448,0.002135,0.013075,0.954077,0.695940,0.258137,0.001683,0.024688,0.002544
70,TF-IDF- [min_df: 5],KNN - [n_neighbors: 3],0.739878,0.617742,0.122136,0.102235,0.011546,0.950730,0.693012,0.257718,0.005090,0.014859,0.002908
